In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/usr/lib/ultralytics_for_offline_install_mine/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_model
import glob

MainProcess


In [3]:
# Construct model
model = flg_model.FinalModel()
model.ratio_of_motors_allowed = 0.45
model.estimate_voxel_spacing = True
model.run_in_parallel = False
model.state = 1
if not fls.env=='kaggle':    
    res_dir = fls.model_dir + '/many_full_res/'
    #fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', res_dir)
else:
    res_dir = '/kaggle/input/byu-many-models/'
files = glob.glob(res_dir+'/*.pickle')
files.sort()
for f in files[:10]:
    data = fls.dill_load(f)
    this_model = data.trained_model
    this_model.step1Labels.preprocessor.voxel_scale = 1.
    model.step1_list.append(this_model.step1Labels)

In [4]:
# Infer
inferred_test_data = model.infer(fls.load_all_test_data())
for d in inferred_test_data:
    print(d.name)
    print(d.labels)
fls.write_submission_file(inferred_test_data)

Clearing pytorch


C:\Users\jeroe\anaconda3\envs\jupyterlab-debugger2\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


Clearing cupy
Prepping
YOLOv8m summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_003acc (1/1)
Processing 500 out of 500 slices (CONCENTRATION=1)
Clearing pytorch
Clearing cupy
Prepping
YOLOv8m summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_003acc (1/1)
Processing 500 out of 500 slices (CONCENTRATION=1)
500
[179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 223 224 225 226 227 228 229 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 358 359 3

In [ ]:
%%time
if not fls.is_submission:
    data = model.infer(fls.load_all_train_data()[::10])
    for d in data:
        print(d.labels)